In [27]:
import json
import os 
from datetime import datetime

def stat(fp):
    with open(fp) as f:
        data = json.load(f)
    df = pd.DataFrame([{'datetime': r['date'], 'bike': r['bike'], 'car': r['car'], 'pedestrian': r['pedestrian']} for r in data['report']])
    if len(df):
        df.index = pd.to_datetime(df['datetime'], format='%Y-%m-%dT%H:%M:%S.000Z')
        del df['datetime']
        df = df.resample('D').sum()
        return dict(df.mean())

In [102]:
from collections import defaultdict
  
data = defaultdict(dict)

idir = 'data/'
for fp in os.listdir(idir):
    try:
        s_id, year, quarter = fp[:-5].split('_')
        data[s_id][f"{year}_{quarter}"] = stat(idir+fp)
    except Exception:
        print("Error with file ", fp)


Error with file  belgium-latest.osm.pbf
Error with file  .DS_Store
Error with file  segments.json
Error with file  dev2
Error with file  dev


In [134]:
for s_id, d in data.items():
    if not all(i is None for i in d.values()):
        print(pd.DataFrame(d).mean(axis=1))
#{s:pd.DataFrame(data[s]).mean(axis=1) for s in data.keys()}
#{s:pd.DataFrame(data[9000000179]).mean(axis=1) for s in data.keys()}


9000000948
False
bike          168.161952
car           417.236615
pedestrian     22.208720
dtype: float64

9000000193
False
bike          117.477677
car           558.473713
pedestrian     23.375730
dtype: float64

9000000905
False
bike           149.661513
car           1384.012174
pedestrian      12.863067
dtype: float64

9000000878
False
bike          126.251599
car           983.510057
pedestrian     13.587714
dtype: float64

9000000189
False
bike           178.113174
car           1352.045843
pedestrian      44.517013
dtype: float64

9000000179
True


ValueError: If using all scalar values, you must pass an index

In [131]:
all(d.values())

False

In [110]:
df.mean(axis=1)

bike          168.161952
car           417.236615
pedestrian     22.208720
dtype: float64

In [ ]:
from collections import defaultdict
  
data = defaultdict([])

results = {fp:stat(idir+fp) for fp in os.listdir(idir)}
print(len(results))
filtered = {k: v for k, v in results.items() if v is not None}
with open('labels.json','w') as f:
    json.dump(filtered, f, indent=2)
print(len(filtered))